<a href="https://colab.research.google.com/github/SaswatD27/21cs460_group01/blob/main/Code/Subclustering_P2P_OldFramework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bootstrapping

In [ ]:
import random
import time

import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import DBSCAN
import os

# Data Handler

### Handle data for training in device and testing in server

In [ ]:
# Generates three set of data with linear trend
def get_data(m, n): 
    x_cord = []
    y_cord = []
    for i in range(n):
        x = np.random.rand(w_n)
        y = np.dot(m, x)
        x_cord.append(x)
        y_cord.append(y)
    return x_cord, y_cord

def data_1():
    m = m1
    # print(m)
    x, y = get_data(m, sum(n_k_dev))
    i = 0
    x_cord = []
    y_cord = []
    for n0 in n_k_dev:
        x_cord.append(x[i:i + n0])
        y_cord.append(y[i:i + n0])
        i += n0
    return x_cord, y_cord


def test_data1():
    m = m1
    testing_x, testing_y = get_data(m, 100)
    return testing_x, testing_y

################################################################################
def data_2():
    m = m2
    # print(m)
    x, y = get_data(m, sum(n_k_dev))
    i = 0
    x_cord = []
    y_cord = []
    for n0 in n_k_dev:
        x_cord.append(x[i:i + n0])
        y_cord.append(y[i:i + n0])
        i += n0
    return x_cord, y_cord


def test_data2():
    m = m2
    testing_x, testing_y = get_data(m, 100)
    return testing_x, testing_y

################################################################################
def data_3():
    m = m3
    # print(m)
    x, y = get_data(m, sum(n_k_dev))
    i = 0
    x_cord = []
    y_cord = []
    for n0 in n_k_dev:
        x_cord.append(x[i:i + n0])
        y_cord.append(y[i:i + n0])
        i += n0
    return x_cord, y_cord


def test_data3():
    m = m3
    testing_x, testing_y = get_data(m, 100)
    return testing_x, testing_y

# Client's Side setup

In [ ]:
# Performs client side SGD locally
def SGD_(m, n, iteration, x_cord, y_cord, alpha):  
    for p in range(iteration):
        for i in range(n):
            j = random.randint(0, n - 1)
            x_i = x_cord[j]
            y_a = y_cord[j]
            y_p = np.dot(m, x_i)
            dm = 2 * x_i * (y_p - y_a)
            m = m - alpha * dm
    return m


def no_of_pts(id):
    return n_k_dev[id]


def fetch_coords(id, x_cord, y_cord):
    return x_cord[id], y_cord[id]


## Different local learnings to replicate training of data with different devices having different model

In [ ]:
def device_local_learning(alp_, w, id, itern):
    # print("Device %i running" % id)
    n_k = no_of_pts(id)
    x_cord, y_cord = data_1()
    x_cord, y_cord = fetch_coords(id, x_cord, y_cord)
    w = SGD_(w, n_k, itern, x_cord, y_cord, alp_)
    save_weight(w, n_k, id)
    return w, n_k


def device_local_learning_2(alp_, w, id, itern):
    # print("Device %i running" % id)
    n_k = no_of_pts(id)
    x_cord, y_cord = data_2()
    x_cord, y_cord = fetch_coords(id, x_cord, y_cord)
    w = SGD_(w, n_k, itern, x_cord, y_cord, alp_)
    save_weight(w, n_k, id)
    return w, n_k


# Conducts local training for SGD and outputs weight vector
def device_local_learning_3(alp_, w, id, itern):
    # print("Device %i running" % id)
    n_k = no_of_pts(id)
    x_cord, y_cord = data_3()
    x_cord, y_cord = fetch_coords(id, x_cord, y_cord)
    w = SGD_(w, n_k, itern, x_cord, y_cord, alp_)
    save_weight(w, n_k, id)
    return w, n_k

# Server Side setup

In [ ]:
#Vanilla Fedrated average for  SGD
def federated_avg(w_, n_k):
    n_ = np.sum(n_k)
    w_k = 0
    for p in range(len(n_k)):
        w_k += (n_k[p] * w_[p]) / n_
    return w_k


def SGD_federated_learning_new(T, K, C, w, n, alp_):
    T1 = T
    w_ = np.zeros((int(C * K), w_n))
    n_k = np.zeros(int(C * K))
    prob = 0.8
    prob2 = 0.95
    # Train for T many rounds(Global Model)
    for i in range(T1):
        # Random devices for traing the model
        training_subset = np.random.choice(all_device, int(C * K), replace=False)
        z = 0
        for j in training_subset:
            # To simullate a random fraction of devices that have differnt data set 
            ran = random.random()
            if ran < prob:
                w_[z], n_k[z] = device_local_learning(alp_, w, j, n)
            elif ran > prob and ran < prob2:
                w_[z], n_k[z] = device_local_learning_2(alp_, w, j, n)
            else:
                w_[z], n_k[z] = device_local_learning_3(alp_, w, j, n)
            z += 1
        w_k = federated_avg(w_, n_k)
        w = w_k
        print("Global Iteration ", i, " done.")


    print("Initial Model sent.")
    w_global = w
    #Sent the weights for clustering
    w_k, device_no = new_update(w_, training_subset, n_k, alp_, n, T)
    return w_k, w_global, device_no


def new_update(w_, training_subset, n_k, alp_, n, T):
    w_mult = []
    # Clustering of devices via their weight vectors
    clustering = DBSCAN(eps=0.5, min_samples=1).fit(w_)
    print("Labels for devices",clustering.labels_)
    device_number = []
    device_nu=0
    #DO for all the labels
    for i in range(np.max(clustering.labels_) + 1):
        training_subset_n = []
        n_k_n = []
        w_new = []
        #Make a subset of traing subset taht have same cluster-id
        for k in range(len(training_subset)):
            lab = clustering.labels_[k]
            if i == lab:
                training_subset_n.append(training_subset[k])
                n_k_n.append(n_k[k])
                w_new.append(w_[k])
                device_nu+=1
        device_number.append(len(training_subset_n))
        #For T many iteration do the P2P traing
        for j in range(T):
            initiate = np.random.choice(training_subset_n)
            w_t = p2p_learn(initiate, training_subset_n, i)
            print("Class ", i, " Iteration ", j, " done.")
        """for i in training_subset_n:
            i = str(i)
            os.remove(i + "_wn.dat")"""
        #Save the models for all calsses
        w_mult.append(w_t)
        print(device_number,device_nu)
    return w_mult, device_number

## Testing of the Model

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]


def average_testing_error(w, testing_x, testing_y):
    error = 0
    for i in range(len(testing_x)):
        y_p = np.dot(w, testing_x[i])
        y_a = testing_y[i]
        # print(i)
        error += (y_a - y_p) ** 2
    error *= (1 / len(testing_x))
    return error

In [ ]:
#Request Other device for their weights
def request_dat(initiate):
    fl = str(initiate)
    w_n = np.loadtxt(fl + "_wn.dat")
    n_k_new = n_k_devices[initiate]
    return w_n, n_k_new

#Initiate P2P learn by making a traing subset which doesnot include the device which is calling
def p2p_learn(initate, training_subset, clas):
    w_n, n_k_new = request_dat(initate)
    training_subset_c = training_subset.copy()
    training_subset_c.remove(initate)
    w_new = communicate(w_n, training_subset_c, n_k_new, clas)
    return w_new

# Initiate communication and does the aggreagtion
def communicate(w_o, traiinng_subset, n_k, clas):
    print(traiinng_subset)
    #Take a random subset of traing subset and call them 
    comm_len = np.random.randint(1, len(traiinng_subset))
    call_to = np.random.choice(traiinng_subset, comm_len)
    print("Call", call_to)

    #Agrregate all the weights available
    w_ = []
    n_ = []
    w_.append(w_o)
    n_.append(n_k)
    for i in call_to:
        w_w, n_w = request_dat(i)
        w_.append(w_w)
        n_.append(n_w)
    w_new = federated_avg(w_, n_)
    
    #Compares old and new weights to decide which one performs better
    if clas == 1:
        testing_x1, testing_y1 = test_data1()
        err_o = average_testing_error(w_o, testing_x1, testing_y1)
        err_1 = average_testing_error(w_new, testing_x1, testing_y1)
    elif clas == 2:
        testing_x2, testing_y2 = test_data2()
        err_o = average_testing_error(w_o, testing_x2, testing_y2)
        err_1 = average_testing_error(w_new, testing_x2, testing_y2)
    else:
        testing_x3, testing_y3 = test_data3()
        err_o = average_testing_error(w_o, testing_x3, testing_y3)
        err_1 = average_testing_error(w_new, testing_x3, testing_y3)
        
    if err_1 < err_o:
        w_nn = w_new
    else:
        w_nn = w_o

    m=0

    #Save new model
    for i in call_to:
        n_k_=n_[m]
        save_weight(w_nn,n_k_,i)
        m+=1

    return w_nn

#For Saving the model/Weights Loacally
def save_weight(w, n_k, id):
    n_k_devices[id] = n_k
    fl = str(id)
    file = open(fl + "_wn.dat", "w")
    w_p = [w]
    for row in w_p:
        np.savetxt(file, row)
    file.close()

In [ ]:
#Three different weights to represent different data
m1 = [1.5, 12, 4.0, 2.9, 4.8, 6.1, 1]
m2 = [1, 12, 4, 3, 5, 6, 1.2]
m3 = [1.1, 12.2, 3.9, 3, 4.8, 5.8, 0.7]
k = 500
n_k_dev = np.random.randint(100, 1000, k)
w_n = 7  # cardinality of weight vector

T = 10  # No of federated learning rounds
K = 500  # Total no of nodes
C = 0.25  # fraction of nodes used at each iteration
# B =K*C #Local batch size used at each learning iteration
# random Model
w = np.random.rand(w_n)
n = 50  # number of iteration for local training before pulling
alp_ = 0.01  # local learning rate
n_k = random.randint(10, 100)
n_k_devices = np.zeros(K)
all_device = np.arange(K)
# x_cord1, y_cord1 = get_data(m, n_k)

time1 = time.time()
w1, w_global, device_no = SGD_federated_learning_new(T, K, C, w, n, alp_)
time2 = time.time()
print("Time taken for SGD federated learning", time2 - time1)

Global Iteration  0  done.
Initial Model sent.
Labels for devices [0 0 1 0 1 0 0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 2 0 1 1 0 0 0 2 0 1 0 0 0 0 0 0
 0 1 2 0 0 0 1 0 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 2 0 0 1 0 0 0 0 2 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[289, 21, 390, 393, 127, 168, 464, 488, 183, 271, 109, 385, 216, 353, 222, 478, 38, 487, 115, 446, 6, 254, 291, 355, 84, 150, 211, 223, 67, 158, 369, 42, 197, 226, 372, 206, 138, 405, 35, 33, 470, 274, 13, 329, 484, 461, 497, 499, 48, 152, 188, 335, 144, 434, 63, 175, 154, 230, 332, 269, 64, 384, 394, 117, 379, 102, 88, 60, 324, 82, 15, 174, 462, 163, 493, 227, 331, 281, 68, 257, 396, 475, 312, 243, 59, 366, 98, 354, 314, 156, 270, 50, 74, 204, 217, 252, 308, 275, 295, 294, 460]
Call [ 50  33 168 150 487 117 332 294 217 152 294 493 308 379 484  63 197 497
 353 487 329 174  35  84 204 163 332 329 222]
Class  0  Iteration  0  done.
[102] 102
[375, 358, 91, 210, 24, 277, 31

In [ ]:
print("Original weigts for minority(I) is ", m1, " and minority(II) ", m2, " and majority is:", m3)
print("Global model has weights:", w_global)
print("Classed multi model has weights:", w1)
print("Clustering number", device_no)

Original weigts for minority(I) is  [1.5, 12, 4.0, 2.9, 4.8, 6.1, 1]  and minority(II)  [1, 12, 4, 3, 5, 6, 1.2]  and majority is: [1.1, 12.2, 3.9, 3, 4.8, 5.8, 0.7]
Global model has weights: [ 1.4054721  12.01300206  3.99349623  2.92021045  4.82740854  6.0667765
  1.00789139]
Classed multi model has weights: [array([ 1.5       , 12.        ,  3.99999999,  2.9       ,  4.79999999,
        6.09999999,  1.00000002]), array([ 1. , 12. ,  4. ,  3. ,  5. ,  6. ,  1.2]), array([ 1.10000491, 12.19999829,  3.89999774,  2.999999  ,  4.80000236,
        5.79999439,  0.70000325])]
Clustering number [102, 15, 8]


In [ ]:
m_1 = w1[0]
m_2 = w1[1]
m_3 = w1[2]
#Testing for different multi models for different data 
testing_x1, testing_y1 = test_data1()
testing_x2, testing_y2 = test_data2()
testing_x3, testing_y3 = test_data3()
print("Total number of device = ", int(K * C))
print("Cluster having the majority has %i devices." % device_no[0])
print("Cluster having the minority has %i devices and %i devices." % (device_no[1], device_no[2]))
print("Average Testing Error for Global model:\n For majority devices error =",
      average_testing_error(w_global, testing_x1, testing_y1), "\n For minority cluster with ", device_no[1],
      " devices has error =", average_testing_error(w_global, testing_x2, testing_y2), "\n For minority cluster with ",
      device_no[2], "devices error =", average_testing_error(w_global, testing_x3, testing_y3))

print("Average Testing Error for Separate classified model:\n For majority devices error",
      average_testing_error(m_3, testing_x3, testing_y3), "\n For minority cluster with ", device_no[1],
      " devices has error =", average_testing_error(m_1, testing_x1, testing_y1), "\n For minority cluster with ",
      device_no[2], "devices error =", average_testing_error(m_2, testing_x2, testing_y2))

Total number of device =  125
Cluster having the majority has 102 devices.
Cluster having the minority has 15 devices and 8 devices.
Average Testing Error for Global model:
 For majority devices error = 0.0022940326985583397 
 For minority cluster with  15  devices has error = 0.018326042559526037 
 For minority cluster with  8 devices error = 0.15510179756883888
Average Testing Error for Separate classified model:
 For majority devices error 5.680703753720265e-12 
 For minority cluster with  15  devices has error = 7.050767717493925e-17 
 For minority cluster with  8 devices error = 2.46557438511628e-22
